In [1]:
import pandas as pd 
from llmlab.sanity_check.syntax.response_to_function import parse_response_to_functions_exp
import llmlab
from collections import defaultdict
import numpy as np 

def merge_content(data):
    """
    Merge the content of a list of dictionaries by combining all unique 'Hardware' and 'Reagent' values.
    
    Parameters:
    data (list): A list of dictionaries where each dictionary contains 'Hardware' (list) and 'Reagent' (list).
    
    Returns:
    dict: A dictionary with merged unique 'Hardware' and 'Reagent' values, excluding None values and ensuring uniqueness.
    """
    # Merging all unique hardware and reagent values
    merged_hardware = set()
    merged_reagent = set()
    
    for entry in data:
        merged_hardware.update(set(entry['Hardware']))
        merged_reagent.update(set(entry['Reagent']))
    
    # Filter out None values from reagents
    merged_reagent.discard(None)
    
    return {'Hardware': (merged_hardware), 'Reagent': (merged_reagent)}

examples = pd.read_json("./examples.json")

In [2]:
# iterate through all the experiments we have done 
xdl_exp = {}
content_exp = {}
# we have up to 66 experiments 
for i in range(66):
    code = examples[f"exp_{i}"]["code"]
    # ignore the failed ones 
    if examples[f"exp_{i}"]["code"] != "syntax_not_passed":
        functions, errors = parse_response_to_functions_exp(examples[f"exp_{i}"]["code"])
        
        # create a list to store the information in this experiment
        xdl_step = []
        content_step = []
        
        # iterate through all the functions 
        for step_temp in functions:
            # if it is a yield step, we ignore it. 
            if type(step_temp) == llmlab.operations.exp_operation.yield_.Yield:
                pass
            else:
            # else, all the steps should been converted
                xdl_step_temp, content_temp = step_temp.to_xdl()
                if xdl_step_temp!=None:
                    print(xdl_step_temp.name)
                    print(xdl_step_temp.properties)
                else:
                    raise ValueError()
                
                # add this experiment to the list 
                xdl_step.append(xdl_step_temp)
                content_step.append(content_temp)
    # add the experiments   
    xdl_exp[f"exp_{i}"] = xdl_step
    content_exp[f"exp_{i}"] = merge_content(content_step)

Add
{'vessel': 'pyrex vials', 'reagent': 'HfOCl2.8H2O', 'volume': None, 'mass': '17.9 mg', 'dropwise': False, 'speed': None, 'time': None, 'stir': False, 'stir_speed': None, 'viscous': False, 'purpose': None, 'amount': '17.9 mg', 'comment': ''}
Add
{'vessel': 'pyrex vials', 'reagent': 'H3BTB', 'volume': None, 'mass': '12.5 mg', 'dropwise': False, 'speed': None, 'time': None, 'stir': False, 'stir_speed': None, 'viscous': False, 'purpose': None, 'amount': '12.5 mg', 'comment': ''}
Add
{'vessel': 'pyrex vials', 'reagent': 'DMF', 'volume': '2.05 mL', 'mass': None, 'dropwise': False, 'speed': None, 'time': None, 'stir': False, 'stir_speed': None, 'viscous': False, 'purpose': None, 'amount': '2.05 mL', 'comment': ''}
Add
{'vessel': 'pyrex vials', 'reagent': 'HCOOH', 'volume': '0.88 mL', 'mass': None, 'dropwise': False, 'speed': None, 'time': None, 'stir': False, 'stir_speed': None, 'viscous': False, 'purpose': None, 'amount': '0.88 mL', 'comment': ''}
Add
{'vessel': 'pyrex vials', 'reagent':

In [3]:
from xdl import XDL
from llmlab.xdl_link.platform import VirtualPlatform
from xdl.hardware import Component, Hardware
from xdl.reagents import Reagent

In [4]:
for exp in list(xdl_exp.keys()):
    xdl_file = XDL("./xdl_template.xdl", platform = VirtualPlatform)
    xdl_file.steps = xdl_exp[exp]

    hardware_entry = np.unique([i for i in content_exp[exp]["Hardware"]])
    reagent_entry = []
    for i in content_exp[exp]["Reagent"]:
        if type(i) == str:
            reagent_entry.append(i)
        else:
            if i.one_identity_with_concentration() != "":
                reagent_entry.append(
                    i.one_identity_with_concentration()
                )
    reagent_entry = np.unique(reagent_entry)
    
    xdl_file.hardware = Hardware([Component(id = i, component_type = "reactor") for i in hardware_entry])
    xdl_file.reagents = [Reagent(i) for i in reagent_entry]
    
    xdl_file.save(f"./{exp}.xdl")
    test = XDL(f"./{exp}.xdl", platform = VirtualPlatform)